Lien de téléchargement des données: https://cvml.ista.ac.at/AwA2/        

13GB file : https://cvml.ista.ac.at/AwA2/AwA2-data.zip

In [2]:
import sys
import cv2 # Pour utiliser open_cv, il faut la version de python est 3.7
import os
import csv

import numpy as np 
import pandas as pd 
import math

import torch 
from torch.utils.data import Dataset, DataLoader
import torchvision 
from torchvision.io import read_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms

np.random.seed(0)

In [24]:
# Constant. Should be the path to the folder named JPEGImages, containing the 33K images in its subfolders.
JPEGIMAGES_FOLDER_PATH = "E:\\3710datas\\Animals_with_Attributes2\\JPEGImages"

In [37]:
# quick test
test = JPEGIMAGES_FOLDER_PATH+"\\fox\\fox_10001.jpg"
img = cv2.imread(test) 
print(img.shape) #ndarray
print(type(img))
cv2.imshow('Sample Image from AwA2 dataset',img)
cv2.waitKey(0)

(764, 918, 3)
<class 'numpy.ndarray'>


-1

In [38]:
labels_dirs = os.listdir(JPEGIMAGES_FOLDER_PATH)
print(labels_dirs)
len(labels_dirs) # 50 labels / subdirectories

['antelope', 'bat', 'beaver', 'blue+whale', 'bobcat', 'buffalo', 'chihuahua', 'chimpanzee', 'collie', 'cow', 'dalmatian', 'deer', 'dolphin', 'elephant', 'fox', 'german+shepherd', 'giant+panda', 'giraffe', 'gorilla', 'grizzly+bear', 'hamster', 'hippopotamus', 'horse', 'humpback+whale', 'killer+whale', 'leopard', 'lion', 'mole', 'moose', 'mouse', 'otter', 'ox', 'persian+cat', 'pig', 'polar+bear', 'rabbit', 'raccoon', 'rat', 'rhinoceros', 'seal', 'sheep', 'siamese+cat', 'skunk', 'spider+monkey', 'squirrel', 'tiger', 'walrus', 'weasel', 'wolf', 'zebra']


50

# Note : Some labels have a low number of images. 

## Possible solutions to explore : 
    Data augmentation : creating new training data by applying random transformations to existing images, such as rotating, cropping, or flipping them.

In [39]:
def find_num_images_per_label(img_dir = JPEGIMAGES_FOLDER_PATH) -> tuple[dict,dict]: 
    """ 
    USEFUL FOR SAMPLING.
    Return a dict with keys as the 50 labels, and values being the number of images in each subdirectory corresponding to label
    and a second dict with the relative numbers (proportion) for every label compared to the total number of images (useful for sampling)"""
    labels_dirs = os.listdir(img_dir)
    num_images_per_label = dict.fromkeys(labels_dirs)
    proportions_images_per_label = dict.fromkeys(labels_dirs)
    total_num_images = 0

    # Update absolute number of images per label
    for i, label in enumerate(labels_dirs) : 
        specific_label_path = os.path.join(img_dir, labels_dirs[i])
        num_images_label = len(os.listdir(specific_label_path))
        total_num_images += num_images_label
        num_images_per_label[label] = num_images_label

    # Update relative number of images per label (proportion)
    for i, label in enumerate(labels_dirs) : 
        num_images_label = num_images_per_label[label]
        proportion_label = round(num_images_label / total_num_images, 4)
        proportions_images_per_label[label] = proportion_label

    return num_images_per_label, proportions_images_per_label

num_images_per_label, proportions_images_per_label = find_num_images_per_label()
print(num_images_per_label)
print(proportions_images_per_label)

{'antelope': 1046, 'bat': 383, 'beaver': 193, 'blue+whale': 174, 'bobcat': 630, 'buffalo': 895, 'chihuahua': 567, 'chimpanzee': 728, 'collie': 1028, 'cow': 1338, 'dalmatian': 549, 'deer': 1344, 'dolphin': 946, 'elephant': 1038, 'fox': 664, 'german+shepherd': 1033, 'giant+panda': 874, 'giraffe': 1202, 'gorilla': 872, 'grizzly+bear': 852, 'hamster': 779, 'hippopotamus': 684, 'horse': 1645, 'humpback+whale': 709, 'killer+whale': 291, 'leopard': 720, 'lion': 1019, 'mole': 100, 'moose': 704, 'mouse': 185, 'otter': 758, 'ox': 728, 'persian+cat': 747, 'pig': 713, 'polar+bear': 868, 'rabbit': 1088, 'raccoon': 512, 'rat': 310, 'rhinoceros': 696, 'seal': 988, 'sheep': 1420, 'siamese+cat': 500, 'skunk': 188, 'spider+monkey': 291, 'squirrel': 1200, 'tiger': 877, 'walrus': 215, 'weasel': 272, 'wolf': 589, 'zebra': 1170}
{'antelope': 0.028, 'bat': 0.0103, 'beaver': 0.0052, 'blue+whale': 0.0047, 'bobcat': 0.0169, 'buffalo': 0.024, 'chihuahua': 0.0152, 'chimpanzee': 0.0195, 'collie': 0.0275, 'cow': 0.

In [17]:
ANNOTATIONS_FILENAME = 'annotations.csv'

def create_annotations_csv_file(annotations_filename = ANNOTATIONS_FILENAME, img_dir = JPEGIMAGES_FOLDER_PATH) : 
    """ 
    Create a csv annotations_file, annotations.csv, with two columns, in the format : 
                        path/to/image, label
    
    The annotation csv is necessary for DataLoader.
    """

    labels_dirs:list = os.listdir(img_dir)
   
    if os.path.exists(annotations_filename):
        os.remove(annotations_filename)
        print(f'Deleted existent {ANNOTATIONS_FILENAME} file.\n ---------------------------')
    
    with open(annotations_filename, 'w', newline='') as file :
        writer = csv.writer(file, dialect='excel', delimiter=',')

        for i, label in enumerate(labels_dirs) : 

            specific_label_path = os.path.join(img_dir, label)
            images_names = os.listdir(specific_label_path)

            for j, image_name in enumerate(images_names):
                full_path_to_img= os.path.join(specific_label_path, image_name)
                full_path_to_img= os.path.join(label, image_name)

                row = [full_path_to_img, label]
                writer.writerow(row)

    print(f'Sucessfully created {ANNOTATIONS_FILENAME} file.')

#
create_annotations_csv_file()

Deleted existent annotations.csv file.
 ---------------------------
Sucessfully created annotations.csv file.


In [18]:
class AWA2Dataset(Dataset): # Dataset class to serve as input for the DataLoader.
    """ 
    Dataset class to serve as input for the DataLoader.
    Implements all the required methods and more. 
    """

    def __init__(self, annotations_file=ANNOTATIONS_FILENAME, img_dir=JPEGIMAGES_FOLDER_PATH, 
                transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

        numbers_infos_dicts: tuple[dict,dict] = find_num_images_per_label(img_dir=JPEGIMAGES_FOLDER_PATH)
        self.num_images_per_label = numbers_infos_dicts[0]
        self.proportions_images_per_label = numbers_infos_dicts[1]

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # img_path = self.img_labels.iloc[idx, 0]
        label = self.img_labels.iloc[idx, 1]

        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [19]:
dataset = AWA2Dataset()

## TODO : Change transforms. Currently this is not useful.
dataset.transform = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    # transforms.ToTensor(), # Already a tensor as implemented in Dataset class with the reaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
                    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ])

# Testing. All good
random_index = np.random.randint(0, len(dataset))
image, label = dataset[random_index]
print(label)
print(image)

buffalo
tensor([[[ 90,  98, 110,  ..., 197, 200, 194],
         [ 77,  73,  97,  ..., 197, 196, 111],
         [ 89,  79, 103,  ..., 195, 172,  69],
         ...,
         [129, 199, 160,  ...,  99, 126,  87],
         [145,  95, 175,  ..., 103,  77, 125],
         [ 97, 117, 191,  ...,  65, 110,  98]],

        [[130, 142, 138,  ..., 231, 229, 225],
         [121, 100, 128,  ..., 229, 231, 125],
         [118, 105, 135,  ..., 231, 191,  76],
         ...,
         [155, 208, 188,  ..., 131, 155, 122],
         [143, 101, 193,  ..., 135, 104, 158],
         [104, 114, 207,  ...,  82, 141, 126]],

        [[ 63,  69, 120,  ..., 255, 252, 248],
         [ 67,  67,  69,  ..., 254, 251, 178],
         [ 82,  79, 101,  ..., 255, 241, 121],
         ...,
         [ 97, 166, 119,  ...,  81, 100,  36],
         [104,  47, 136,  ...,  84,  34,  48],
         [ 58,  66, 141,  ...,  43,  63,  58]]], dtype=torch.uint8)


In [25]:
# Experiment with DataLoader. Everything works good
dataloader = DataLoader(dataset = dataset, batch_size=4, shuffle=True)
dataiter = iter(dataloader)
data = next(dataiter)

images, labels = data 
print(labels, images)

('horse', 'giraffe', 'hippopotamus', 'tiger') tensor([[[[226, 227, 226,  ..., 225, 224, 224],
          [227, 224, 228,  ..., 226, 223, 223],
          [226, 229, 225,  ..., 225, 223, 221],
          ...,
          [170, 137, 167,  ..., 171, 169, 187],
          [220, 227, 220,  ..., 160, 229, 228],
          [208, 208, 217,  ..., 180, 163, 188]],

         [[232, 232, 231,  ..., 225, 225, 225],
          [231, 228, 232,  ..., 225, 227, 227],
          [232, 230, 229,  ..., 224, 226, 224],
          ...,
          [165, 131, 166,  ..., 169, 168, 184],
          [219, 227, 217,  ..., 156, 229, 220],
          [207, 207, 218,  ..., 176, 158, 185]],

         [[233, 232, 232,  ..., 229, 228, 229],
          [233, 229, 233,  ..., 230, 230, 230],
          [233, 234, 230,  ..., 230, 229, 229],
          ...,
          [165, 141, 172,  ..., 164, 160, 178],
          [214, 221, 211,  ..., 145, 221, 217],
          [202, 204, 213,  ..., 173, 153, 179]]],


        [[[111, 115, 102,  ...,  70, 

In [31]:
# Training loop example
num_epochs = 2 
batch_size = 4
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/batch_size)
print(total_samples, n_iterations)

dataloader = DataLoader(dataset = dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs) : 
    # loop over trainloader 
    for i, (inputs, labels) in enumerate(dataloader) : 
        
        # Do forward and backward pass, update the weights 
        if(i+1) % 5 == 0 :
            print(f'epoch {epoch+1} / {num_epochs}, step, {i+1}/{n_iterations}, inputs {inputs.shape}')

        if i==20 : 
            print('Completed')
            break

37321 9331
epoch 1 / 2, step, 5/9331, inputs torch.Size([4, 3, 224, 224])
epoch 1 / 2, step, 10/9331, inputs torch.Size([4, 3, 224, 224])
epoch 1 / 2, step, 15/9331, inputs torch.Size([4, 3, 224, 224])
epoch 1 / 2, step, 20/9331, inputs torch.Size([4, 3, 224, 224])
Completed
epoch 2 / 2, step, 5/9331, inputs torch.Size([4, 3, 224, 224])
epoch 2 / 2, step, 10/9331, inputs torch.Size([4, 3, 224, 224])
epoch 2 / 2, step, 15/9331, inputs torch.Size([4, 3, 224, 224])
epoch 2 / 2, step, 20/9331, inputs torch.Size([4, 3, 224, 224])
Completed
